In [1]:
import pandas as pd
import csv
from transformers import MT5Tokenizer
import os
from collections import Counter
from tqdm.notebook import tqdm
from glob import glob

In [ ]:
for tsv in glob("../../defgen/russian/*.gz"):
    data = pd.read_csv(tsv,
        sep='\t', quoting=csv.QUOTE_NONE, compression="gzip"
)
    data.to_csv(f"../../defgen/russian/beam/{os.path.split(tsv)[-1]}",
        sep='\t', quoting=csv.QUOTE_NONE, compression="gzip", header=None, index=None)

In [25]:
data = pd.read_csv(
        "../../defgen/cluster-projects-nn9851k-corpora-diachronic-acl_data-english-corpus1.tsv.gz",
        sep='\t', quoting=csv.QUOTE_NONE, compression="gzip", header=None,
)
data.shape

(27019, 3)

In [26]:
data_diversed = pd.read_csv(
        "../../defgen/cluster-projects-nn9851k-andreku-defgen_lscd-acl_data-english-corpora-acl-parsed-english-corpus1.tsv.gz",
        sep='\t', quoting=csv.QUOTE_NONE, compression="gzip", header=None,
)
data_diversed.shape

(27018, 3)

In [5]:
tokenizer = MT5Tokenizer.from_pretrained(os.path.expanduser("~/mt0-xl"))

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [29]:
bad = data[(data[0]=="face") & data[1].str.contains("Coppe")][1][15030]

In [32]:
len(tokenizer(bad, return_token_type_ids=False, return_attention_mask=False)["input_ids"])

367

In [33]:
data.drop(15030, inplace=True)
data[(data[0]=="face") & data[1].str.contains("Coppe")][1]

Series([], Name: 1, dtype: object)

In [35]:
data.to_csv("../../defgen/greedy/english-corpus1.tsv.gz",
        sep='\t', quoting=csv.QUOTE_NONE, compression="gzip", header=None, index=None)

In [36]:
data.shape

(27018, 3)

In [40]:
data = pd.read_csv(
        "../../defgen/cluster-projects-nn9851k-corpora-diachronic-acl_data-english-corpus2.tsv.gz",
        sep='\t', quoting=csv.QUOTE_NONE, compression="gzip", header=None,
)
print(data.shape)
data_diversed = pd.read_csv(
        "../../defgen/cluster-projects-nn9851k-andreku-defgen_lscd-acl_data-english-corpora-acl-parsed-english-corpus2.tsv.gz",
        sep='\t', quoting=csv.QUOTE_NONE, compression="gzip", header=None,
)
print(data_diversed.shape)

(30903, 3)
(30902, 3)


In [41]:
unique_diversed = data_diversed[1].unique()
for i, row in data.iterrows():
    if row[1] not in unique_diversed:
        print(row.to_dict())
        print(i)
        break
data.shape

{0: 'prop', 1: '381 **26;25353 ; 599 , 712 ** 76;25381 ; Biotech Dev Cp **26;25459 ; A52 , 521 **26;25487 ; 16 , 876 ** 76;25515 ; Bird Inc ** 26;25593 ; A84 , 347 **26;25621 ; 53 , 153 ** 76;25649 ; ** 76;25727 ; Birmingham Steel Cp ** 26;25805 ; A133 , 016 **26;25833 ; 63 , 461 ** 76;25861 ; Blyvooruitzicht Adr ** 26;25939 ; A29 , 781 **26;25967 ; 18 , 103 **76;25995 ; Boatmens Bcshs ** 26;26073 ; R52 , 932 **26;26101 ; 32 , 562 **76;26129 ; Bob Evans Farms Inc ** 26;26207 ; 58 , 102 **26;26235 ; 40 , 094 ** 76;26263 ; Bobbie Brooks Inc ** 26;26341 ; 27 , 555 **26;26369 ; 8 , 239 **76;26397 ; Brajdas Cp ** 26;26475 ; 13 , 689 ** 26;26503 ; 2 , 524 ** 76;26531 ; Brand Cos Inc The ** 26;26609 ; 66 , 721 **26;26637 ; 21 , 788 **76;26665 ; Bristol Gaming Cp ** 26;26743 ; 11 , 383 **26;26771 ; 0 **76;26799 ; Broken Hill Prop Adr ** 26;26877 ; A402 , 398 **26;26905 ; 42 What is the definition of prop?', 2: 'An administrative district.'}
26024


(30903, 3)

In [42]:
data.drop(26024, inplace=True)
data[(data[0]=="prop") & data[1].str.contains("381")][1]

Series([], Name: 1, dtype: object)

In [43]:
data.to_csv("../../defgen/greedy/english-corpus2.tsv.gz",
        sep='\t', quoting=csv.QUOTE_NONE, compression="gzip", header=None, index=None)

In [20]:
max_contexts = 1000
random_state = 42

In [162]:
def cut_frequent(path, out_path):
    data = pd.read_csv(
        path,
        sep='\t', quoting=csv.QUOTE_NONE, compression="gzip", header=None)
    data = data.drop_duplicates()
    new_data = pd.DataFrame()
    for word, n_contexts in Counter(data[0]).items():
        this_word = data[data[0] == word]
        if n_contexts > max_contexts:
            this_word = this_word.sample(max_contexts, random_state=random_state)
        new_data = pd.concat((new_data, this_word))
    new_data = new_data.drop_duplicates()
    print(new_data.shape)
    
    new_data.to_csv(out_path, index=False, sep="\t", compression="gzip")
    print(Counter(new_data[0]))
    return new_data.head()

In [160]:
cut_frequent("../../acl_data/norwegian1-corpus2.txt.gz", "../../acl_data/norwegian1/norwegian1-corpus2.txt.gz")

(24873, 2)


,0,1
22512,forhold,Veksttakten for tjenesteeksporten var i 1990 k...
506496,forhold,Biblioteket har et formelt og meget godt samar...
110510,forhold,I det perspektiv har vi så drøftet forholdet m...
460046,forhold,Videre vil ikke systemet gi noen garanti for e...
244547,forhold,Mest relevant informasjon om verdiskapningen v...


In [163]:
cut_frequent("../../acl_data/norwegian1-corpus1.txt.gz", "../../acl_data/norwegian1/norwegian1-corpus1.txt.gz")

(21799, 2)
Counter({'land': 1000, 'kart': 1000, 'leilighet': 1000, 'tro': 1000, 'mål': 1000, 'mening': 1000, 'bit': 1000, 'formiddag': 1000, 'forhold': 1000, 'stoff': 1000, 'leder': 1000, 'fag': 1000, 'katt': 993, 'rad': 992, 'bølge': 843, 'tropp': 810, 'egg': 598, 'sete': 533, 'damp': 521, 'varsel': 463, 'operasjon': 423, 'gress': 397, 'frakk': 364, 'ris': 326, 'data': 325, 'rev': 304, 'kjemi': 303, 'linse': 291, 'horisont': 284, 'lyng': 208, 'kloakk': 123, 'fil': 109, 'husrom': 105, 'skjerm': 92, 'idiot': 81, 'ørret': 79, 'etterforskning': 61, 'plattform': 61, 'anfektelse': 58, 'pære': 52})


,0,1
36433,land,Undervisningen i geografi har til oppgave å gi...
28711,land,I det hele tatt er jeg overbevist om at dette ...
74769,land,Under storkrigen laget tyokerne flere sandtag ...
72117,land,Som nevnt under punkt C utdannes det her i lan...
8270,land,Overtor Moskva kunde det ka git et tottXBte «o...


In [178]:
data = pd.read_csv(
    "../../acl_data/home-m-corpora-acl-parsed-norwegian2-corpus2.conllu.gz.txt.gz",
    sep='\t', quoting=csv.QUOTE_NONE, compression="gzip", header=None)
Counter(data[0])

Counter({'side': 177657,
         'melding': 136098,
         'bank': 71449,
         'spill': 70953,
         'nett': 32783,
         'rådgiver': 23978,
         'strøm': 23936,
         'vindu': 22508,
         'kanal': 20994,
         'terror': 20147,
         'tape': 19964,
         'virkelighet': 17835,
         'tendens': 16420,
         'papir': 15236,
         'innstilling': 14982,
         'vert': 12113,
         'test': 10997,
         'storby': 8714,
         'kreft': 7974,
         'oppvarming': 6976,
         'melk': 5919,
         'sky': 5630,
         'overflate': 5095,
         'kode': 4902,
         'moment': 3666,
         'virus': 2951,
         'mus': 2520,
         'særforbund': 2167,
         'tjener': 1462,
         'bystyrerepresentant': 1449,
         'fjær': 1372,
         'tavle': 1103,
         'utakt': 989,
         'Syden': 955,
         'stryk': 926,
         'mappe': 386,
         'fane': 374,
         'kjøpepress': 361,
         'driftsregnskap': 24,
  

In [179]:
data = data.drop_duplicates()

In [180]:
new_data = pd.DataFrame()
for word, n_contexts in Counter(data[0]).items():
    this_word = data[data[0] == word]
    if n_contexts > max_contexts:
        this_word = this_word.sample(max_contexts, random_state=random_state)
    new_data = pd.concat((new_data, this_word))
new_data = new_data.drop_duplicates()
print(new_data.shape)

new_data.to_csv("../../acl_data/norwegian2/norwegian2-corpus2.txt.gz", index=False, sep="\t", compression="gzip")
new_data.head()

(35300, 2)


,0,1
699962,rådgiver,Ny Trump-rådgiver går av Tom Bossert trekker s...
590808,rådgiver,Stoknes har jobbet som rådgiver for Olje- og e...
385219,rådgiver,I august tok jeg mot til meg og gikk til skole...
717078,rådgiver,Administrasjonen valgte imidlertid å ikke innf...
737104,rådgiver,Trolig vil KrF få en omtrent like stor stab so...


In [21]:
data = pd.read_csv(
    "../../acl_data/home-m-corpora-acl-parsed-norwegian2-corpus1.conllu.gz.txt.gz",
    sep='\t', quoting=csv.QUOTE_NONE, compression="gzip", header=None)
data = data.drop_duplicates()
new_data = pd.DataFrame()
for word, n_contexts in Counter(data[0]).items():
    this_word = data[data[0] == word]
    if n_contexts > max_contexts:
        this_word = this_word.sample(max_contexts, random_state=random_state)
    new_data = pd.concat((new_data, this_word))
new_data = new_data.drop_duplicates()
print(new_data.shape)
#new_data.to_csv("../../acl_data/norwegian2/norwegian2-corpus1.txt.gz", index=False, sep="\t", compression="gzip")
new_data.head()

(15909, 2)


,0,1
0,vindu,Levittneten tar et normalt vendepunkt et lite ...
15,vindu,Avisa ga meg en rolig oppstemthet og latterkic...
17,vindu,Jeg llvktet ut av mønsteret til Lillehammer - ...
28,vindu,Foran meg - ut av vinduet - kan jeg se tindren...
38,vindu,"Jeg husker mor, far, Hermansen, Nusse, Zappa o..."


In [24]:
new_data.iloc[0][1]

'Levittneten tar et normalt vendepunkt et lite sveklikk og KonBtaterer at det Btar noe lvBende og/eller en lampe pa nGvre bordkant, samtidig klinger muBikken lant2BtiBk, menB l)lik- ket er rettet framover ut av vinduet mot grålysningens ødeleggende effekt. Smale lysflash beveger seg inn på netthinnen. Hva betyr vindu?'

## Русский

In [174]:
targets = pd.read_csv("../../gloss-annotator/wugs/rushifteval_public/annotated_all.tsv", sep="\t", header=None)
targets.head()

,0,1,2,3
0,авторитет,3.233333,2.955556,2.844444
1,амбиция,3.111111,3.444444,3.333333
2,апостол,3.494118,3.426966,3.423529
3,благодарность,3.233333,3.566667,3.655556
4,блин,3.213483,1.662921,2.577778


In [168]:
data = pd.read_csv(
    "../../acl_data/home-m-corpora-acl-parsed-russian-corpus1.conllu.gz.txt.gz",
    sep='\t', quoting=csv.QUOTE_NONE, compression="gzip", header=None)
data = data.drop_duplicates()
print(data.shape)
data = data[data[0].isin(targets[0])]
print(data.shape)


(2123887, 2)
(433737, 2)


In [169]:
 Counter(data[0])

Counter({'земля': 42819,
         'брат': 33900,
         'любовь': 29915,
         'стол': 26135,
         'путь': 25874,
         'правда': 25861,
         'дух': 23903,
         'стена': 17287,
         'век': 14813,
         'огонь': 12666,
         'дядя': 9936,
         'грех': 9190,
         'план': 8568,
         'хозяйка': 7578,
         'завод': 7236,
         'карман': 7206,
         'университет': 6542,
         'название': 5946,
         'расчет': 4689,
         'возраст': 4379,
         'поэзия': 4338,
         'благодарность': 4136,
         'центр': 4038,
         'поколение': 3841,
         'четверть': 3630,
         'облако': 3590,
         'крыша': 3586,
         'памятник': 3574,
         'помощник': 3459,
         'живот': 3074,
         'хор': 2941,
         'полоса': 2893,
         'штаб': 2835,
         'список': 2715,
         'совершенство': 2653,
         'головка': 2316,
         'железо': 2272,
         'цифра': 2255,
         'спутник': 2246,
         'раз

In [170]:
new_data = pd.DataFrame()
for word, n_contexts in Counter(data[0]).items():
    this_word = data[data[0] == word]
    if n_contexts > max_contexts:
        this_word = this_word.sample(max_contexts, random_state=random_state)
    new_data = pd.concat((new_data, this_word))
new_data = new_data.drop_duplicates()
print(new_data.shape)
Counter(new_data[0])

(74374, 2)


Counter({'брат': 1000,
         'облако': 1000,
         'четверть': 1000,
         'возраст': 1000,
         'любовь': 1000,
         'стена': 1000,
         'университет': 1000,
         'завод': 1000,
         'благодарность': 1000,
         'поэзия': 1000,
         'установление': 1000,
         'огонь': 1000,
         'дух': 1000,
         'правда': 1000,
         'памятник': 1000,
         'путь': 1000,
         'план': 1000,
         'век': 1000,
         'дядя': 1000,
         'стол': 1000,
         'земля': 1000,
         'крыша': 1000,
         'грех': 1000,
         'помощник': 1000,
         'ссылка': 1000,
         'частица': 1000,
         'карман': 1000,
         'промышленность': 1000,
         'формула': 1000,
         'совершенство': 1000,
         'железо': 1000,
         'головка': 1000,
         'понедельник': 1000,
         'итальянец': 1000,
         'название': 1000,
         'расчет': 1000,
         'цензура': 1000,
         'заблуждение': 1000,
         'хор':

In [172]:
new_data.to_csv("../../acl_data/russian/russian-corpus1.txt.gz", index=False, sep="\t", compression="gzip")

In [ ]:
data = pd.read_csv(
    "../../acl_data/home-m-corpora-acl-parsed-russian-corpus2.conllu.gz.txt.gz",
    sep='\t', quoting=csv.QUOTE_NONE, compression="gzip", header=None)
data = data.drop_duplicates()
data = data[data[0].isin(targets[0])]
new_data = pd.DataFrame()
for word, n_contexts in Counter(data[0]).items():
    this_word = data[data[0] == word]
    if n_contexts > max_contexts:
        this_word = this_word.sample(max_contexts, random_state=random_state)
    new_data = pd.concat((new_data, this_word))
new_data = new_data.drop_duplicates()
print(new_data.shape)
new_data.to_csv("../../acl_data/russian/russian-corpus2.txt.gz", index=False, sep="\t", compression="gzip")

In [ ]:
data = pd.read_csv(
    "../../acl_data/home-m-corpora-acl-parsed-russian-corpus3.conllu.gz.txt.gz",
    sep='\t', quoting=csv.QUOTE_NONE, compression="gzip", header=None)
data = data.drop_duplicates()
data = data[data[0].isin(targets[0])]
counted = Counter(data[0])
new_data = pd.DataFrame()
for word, n_contexts in counted.items():
    this_word = data[data[0] == word]
    if n_contexts > max_contexts:
        this_word = this_word.sample(max_contexts, random_state=random_state)
    new_data = pd.concat((new_data, this_word))
new_data = new_data.drop_duplicates()
print(new_data.shape)
new_data.to_csv("../../acl_data/russian/russian-corpus3.txt.gz", index=False, sep="\t", compression="gzip")